In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
from sklearn import datasets
from sklearn import metrics as m
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
from itertools import combinations
from sklearn import datasets
from sklearn import metrics as m
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
from itertools import combinations
from sklearn.svm import SVC
import xgboost as xgb
from thundersvm import SVC as svmgpu
from sklearn.utils import shuffle
import itertools
from sklearn.ensemble import VotingClassifier

<IPython.core.display.Javascript object>

In [3]:
randomseed = 10
np.random.seed(randomseed)

xtest = np.array(pd.read_csv("../dataset/xtest.txt"))
xtrain = np.array(pd.read_csv("../dataset/xtrain.txt"))
ytest_original = np.array(pd.read_csv("../dataset/ytest.txt")).ravel()
ytrain_original = np.array(pd.read_csv("../dataset/ytrain.txt")).ravel()

<IPython.core.display.Javascript object>

In [4]:
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()


clf = []
acc = []
ypredproba_all = []
ypredconfprob_all = []
acc_train = []

<IPython.core.display.Javascript object>

1. Baseline classifiers
==============

In [5]:
# classifier 1
# ===========================

rf=RandomForestClassifier(random_state=randomseed, n_estimators=100)
rf.fit(xtrain,ytrain)
rfpred=rf.predict(xtest)
print(m.accuracy_score(ytest,rfpred)) 

clf.append(rf)
acc.append(m.accuracy_score(ytest,rfpred))
ypredproba_all.append(rf.predict_proba(xtest)) 

confmat=m.confusion_matrix(ytest,rfpred)
confsumh=np.sum(confmat,axis=0)
propconfmat=confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:,i]= 100*propconfmat[:,i]/confsumh[i] 
ypredconfprob_all.append(propconfmat/100)


0.7083333333333334


<IPython.core.display.Javascript object>

In [6]:
# classifier 2
# ===========================
svc = svmgpu(random_state=randomseed, probability=True, C=1)
svc.fit(xtrain, ytrain)

svcpred = svc.predict(xtest)
print(m.accuracy_score(ytest, svcpred))

clf.append(svc)
acc.append(m.accuracy_score(ytest, svcpred))
ypredproba_all.append(1 - svc.predict_proba(xtest))

confmat = m.confusion_matrix(ytest, svcpred)
confsumh = np.sum(confmat, axis=1)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[i] = 100 * propconfmat[i] / confsumh[i]
ypredconfprob_all.append(propconfmat / 100)

0.7266666666666667


<IPython.core.display.Javascript object>

In [7]:
# classifier 3
# ===========================
xgbc=xgb.XGBClassifier(random_state=randomseed,n_estimators=10)
xgbc.fit(xtrain,ytrain)

xgbpred=xgbc.predict(xtest)
print(m.accuracy_score(ytest,xgbpred))


clf.append(xgbc)
acc.append(m.accuracy_score(ytest,xgbpred))
ypredproba_all.append(xgbc.predict_proba(xtest))

confmat=m.confusion_matrix(ytest,xgbpred)
confsumh=np.sum(confmat,axis=1)
propconfmat=confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[i]= 100*propconfmat[i]/confsumh[i] 
ypredconfprob_all.append(propconfmat/100)


0.7266666666666667


<IPython.core.display.Javascript object>

2. Voting Classifier 
====

In [8]:
# 2.1 soft voting classfier


eclf3 = VotingClassifier(
    estimators=[("rf", rf), ("svc", svc), ("xgbc", xgbc)],
    voting="soft",
    weights=[1, 1, 1],
    flatten_transform=True,
)
eclf3 = eclf3.fit(xtrain, ytrain)
print(m.accuracy_score(ytest, eclf3.predict(xtest)))

0.715


<IPython.core.display.Javascript object>

In [9]:
acc = []
weight_can = np.arange(0.1, 1, 0.1)
perm = list(itertools.permutations(weight_can, 3))
perm

[(0.1, 0.2, 0.30000000000000004),
 (0.1, 0.2, 0.4),
 (0.1, 0.2, 0.5),
 (0.1, 0.2, 0.6),
 (0.1, 0.2, 0.7000000000000001),
 (0.1, 0.2, 0.8),
 (0.1, 0.2, 0.9),
 (0.1, 0.30000000000000004, 0.2),
 (0.1, 0.30000000000000004, 0.4),
 (0.1, 0.30000000000000004, 0.5),
 (0.1, 0.30000000000000004, 0.6),
 (0.1, 0.30000000000000004, 0.7000000000000001),
 (0.1, 0.30000000000000004, 0.8),
 (0.1, 0.30000000000000004, 0.9),
 (0.1, 0.4, 0.2),
 (0.1, 0.4, 0.30000000000000004),
 (0.1, 0.4, 0.5),
 (0.1, 0.4, 0.6),
 (0.1, 0.4, 0.7000000000000001),
 (0.1, 0.4, 0.8),
 (0.1, 0.4, 0.9),
 (0.1, 0.5, 0.2),
 (0.1, 0.5, 0.30000000000000004),
 (0.1, 0.5, 0.4),
 (0.1, 0.5, 0.6),
 (0.1, 0.5, 0.7000000000000001),
 (0.1, 0.5, 0.8),
 (0.1, 0.5, 0.9),
 (0.1, 0.6, 0.2),
 (0.1, 0.6, 0.30000000000000004),
 (0.1, 0.6, 0.4),
 (0.1, 0.6, 0.5),
 (0.1, 0.6, 0.7000000000000001),
 (0.1, 0.6, 0.8),
 (0.1, 0.6, 0.9),
 (0.1, 0.7000000000000001, 0.2),
 (0.1, 0.7000000000000001, 0.30000000000000004),
 (0.1, 0.7000000000000001, 0.4),
 (0.

<IPython.core.display.Javascript object>

In [10]:
for i in perm:
    print(i)
    eclf3 = VotingClassifier(
        estimators=[("rf", rf), ("svc", svc), ("xgbc", xgbc)],
        voting="soft",
        weights=i,
        flatten_transform=True,
    )
    eclf3 = eclf3.fit(xtrain, ytrain)
    _acc = m.accuracy_score(ytest, eclf3.predict(xtest))
    acc.append(_acc)
    print(_acc)

(0.1, 0.2, 0.30000000000000004)
0.7033333333333334
(0.1, 0.2, 0.4)
0.7233333333333334
(0.1, 0.2, 0.5)
0.7266666666666667
(0.1, 0.2, 0.6)
0.7283333333333334
(0.1, 0.2, 0.7000000000000001)
0.725
(0.1, 0.2, 0.8)
0.7333333333333333
(0.1, 0.2, 0.9)
0.7283333333333334
(0.1, 0.30000000000000004, 0.2)
0.27
(0.1, 0.30000000000000004, 0.4)
0.5866666666666667
(0.1, 0.30000000000000004, 0.5)
0.6966666666666667
(0.1, 0.30000000000000004, 0.6)
0.7133333333333334
(0.1, 0.30000000000000004, 0.7000000000000001)
0.7166666666666667
(0.1, 0.30000000000000004, 0.8)
0.7183333333333334
(0.1, 0.30000000000000004, 0.9)
0.725
(0.1, 0.4, 0.2)
0.26
(0.1, 0.4, 0.30000000000000004)
0.265
(0.1, 0.4, 0.5)
0.32
(0.1, 0.4, 0.6)
0.61
(0.1, 0.4, 0.7000000000000001)
0.69
(0.1, 0.4, 0.8)
0.7066666666666667
(0.1, 0.4, 0.9)
0.7083333333333334
(0.1, 0.5, 0.2)
0.245
(0.1, 0.5, 0.30000000000000004)
0.25666666666666665
(0.1, 0.5, 0.4)
0.26
(0.1, 0.5, 0.6)
0.2866666666666667
(0.1, 0.5, 0.7000000000000001)
0.365
(0.1, 0.5, 0.8)
0.

0.34833333333333333
(0.4, 0.6, 0.5)
0.5866666666666667
(0.4, 0.6, 0.7000000000000001)
0.7
(0.4, 0.6, 0.8)
0.7116666666666667
(0.4, 0.6, 0.9)
0.7183333333333334
(0.4, 0.7000000000000001, 0.1)
0.27166666666666667
(0.4, 0.7000000000000001, 0.2)
0.2733333333333333
(0.4, 0.7000000000000001, 0.30000000000000004)
0.285
(0.4, 0.7000000000000001, 0.5)
0.3566666666666667
(0.4, 0.7000000000000001, 0.6)
0.4866666666666667
(0.4, 0.7000000000000001, 0.8)
0.675
(0.4, 0.7000000000000001, 0.9)
0.6983333333333334
(0.4, 0.8, 0.1)
0.265
(0.4, 0.8, 0.2)
0.2683333333333333
(0.4, 0.8, 0.30000000000000004)
0.27166666666666667
(0.4, 0.8, 0.5)
0.29
(0.4, 0.8, 0.6)
0.31333333333333335
(0.4, 0.8, 0.7000000000000001)
0.37333333333333335
(0.4, 0.8, 0.9)
0.615
(0.4, 0.9, 0.1)
0.2633333333333333
(0.4, 0.9, 0.2)
0.265
(0.4, 0.9, 0.30000000000000004)
0.2683333333333333
(0.4, 0.9, 0.5)
0.27166666666666667
(0.4, 0.9, 0.6)
0.2783333333333333
(0.4, 0.9, 0.7000000000000001)
0.2966666666666667
(0.4, 0.9, 0.8)
0.325
(0.5, 0.1

0.7233333333333334
(0.8, 0.1, 0.7000000000000001)
0.7283333333333334
(0.8, 0.1, 0.9)
0.735
(0.8, 0.2, 0.1)
0.7066666666666667
(0.8, 0.2, 0.30000000000000004)
0.7183333333333334
(0.8, 0.2, 0.4)
0.7183333333333334
(0.8, 0.2, 0.5)
0.7283333333333334
(0.8, 0.2, 0.6)
0.7316666666666667
(0.8, 0.2, 0.7000000000000001)
0.7366666666666667
(0.8, 0.2, 0.9)
0.735
(0.8, 0.30000000000000004, 0.1)
0.71
(0.8, 0.30000000000000004, 0.2)
0.7116666666666667
(0.8, 0.30000000000000004, 0.4)
0.7183333333333334
(0.8, 0.30000000000000004, 0.5)
0.7233333333333334
(0.8, 0.30000000000000004, 0.6)
0.7266666666666667
(0.8, 0.30000000000000004, 0.7000000000000001)
0.73
(0.8, 0.30000000000000004, 0.9)
0.7316666666666667
(0.8, 0.4, 0.1)
0.7066666666666667
(0.8, 0.4, 0.2)
0.71
(0.8, 0.4, 0.30000000000000004)
0.715
(0.8, 0.4, 0.5)
0.715
(0.8, 0.4, 0.6)
0.7166666666666667
(0.8, 0.4, 0.7000000000000001)
0.7266666666666667
(0.8, 0.4, 0.9)
0.7316666666666667
(0.8, 0.5, 0.1)
0.705
(0.8, 0.5, 0.2)
0.7133333333333334
(0.8, 0.5

<IPython.core.display.Javascript object>

In [11]:
np.max(acc)

0.745

<IPython.core.display.Javascript object>

In [12]:
perm[np.argmax(acc)]

(0.4, 0.30000000000000004, 0.9)

<IPython.core.display.Javascript object>